# Chapter 8: Metaprogramming

This notebook contains the sample source code explained in the book *Hands-On Julia Programming, Sambit Kumar Dash, 2021, bpb Publications. All Rights Reserved*.

## 8.1 Introduction

Abstract syntax trees are hierarchical representations of the Julia programs. They ASTs themselves are Julia objects that can be manipulated by the evaluation engine. This is in essence the architecture of the Julia metaprogramming environment. 

### Abstract Syntax Trees (AST)

In [1]:
s = "1+1"

"1+1"

In [2]:
ex = Meta.parse(s)

:(1 + 1)

In [3]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [4]:
eval(ex)

2

In [5]:
ex.args[1] = :(-)
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol -
    2: Int64 1
    3: Int64 1


In [6]:
eval(ex)

0

### Symbols and Interned Strings

In [7]:
Symbol("A", "_", 5, "32")

:A_532

In [8]:
:A_532

:A_532

In [9]:
:(==)

:(==)

In [10]:
ex = :(z = 1 + 2)

:(z = 1 + 2)

In [11]:
dump(ex)

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol z
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 2


In [12]:
eval(ex)

3

In [13]:
z

3

###  Inline Evaluations

In [14]:
x = 5
ex = :($x + 1)

:(5 + 1)

In [15]:
ex = :($(x*x)+1)

:(25 + 1)

In [16]:
y = eval(ex)

26

### Multiline Expressions

In [17]:
ex = quote
    x = 1
    y = 2
    x + y
end

quote
    #= In[17]:2 =#
    x = 1
    #= In[17]:3 =#
    y = 2
    #= In[17]:4 =#
    x + y
end

In [18]:
dump(ex)

Expr
  head: Symbol block
  args: Array{Any}((6,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[17]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol x
        2: Int64 1
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[17]
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol y
        2: Int64 2
    5: LineNumberNode
      line: Int64 4
      file: Symbol In[17]
    6: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Symbol x
        3: Symbol y


In [19]:
ex.args[2].args[1]=:z

:z

In [20]:
ex

quote
    #= In[17]:2 =#
    z = 1
    #= In[17]:3 =#
    y = 2
    #= In[17]:4 =#
    x + y
end

In [21]:
eval(ex)

7

In [22]:
z

1

### Nested Quote and Interpolation

The interpolations are matched with the number of `quote` blocks. Thus two interpolation operations may be needed for the code embedded between two `quote` blocks. 

In [23]:
e = :(1 + 1)

:(1 + 1)

In [24]:
eval(:e)

:(1 + 1)

In [25]:
eval(e)

2

In [26]:
e = :(1+2)
ex = quote 
    quote 
        $e 
    end 
end

quote
    #= In[26]:3 =#
    $(Expr(:quote, quote
    #= In[26]:4 =#
    $(Expr(:$, :e))
end))
end

In [27]:
eval(ex)

quote
    #= In[26]:4 =#
    1 + 2
end

In [28]:
ex = quote 
    quote
       $$e
    end
end

quote
    #= In[28]:2 =#
    $(Expr(:quote, quote
    #= In[28]:3 =#
    $(Expr(:$, :(1 + 2)))
end))
end

In [29]:
eval(ex)

quote
    #= In[28]:3 =#
    3
end

### Functions

Functions can return the interned expressions but cannot evaluate the same. Additional, `eval` calls may be needed. 

In [30]:
function math_exp(op, p1, p2)
    p1f, p2f = map(x-> x isa Number ? 2x : 
                   error("Parameters have to be numbers"), 
                   (p1, p2))
    return Expr(:call, op, p1f, p2f)
end
ex = math_exp(:+, 2, 3)

:(4 + 6)

In [31]:
eval(ex)

10

## 8.2 Macros

Macros computes the interned expression and also calls `eval` on the final expression. Thus, they are convenient than functions. 

In [32]:
macro sayhello()
    return :(println("Hello, World!!!"))
end

@sayhello (macro with 1 method)

In [33]:
@sayhello()

Hello, World!!!


In [34]:
macro sayhello(name)
    return :(println("Hello, ", $name, "!!!"))
end

@sayhello (macro with 2 methods)

In [35]:
@sayhello("John")

Hello, John!!!


### Calling Convention

Macros can be invoked without any brackets surrounding the parameters. 

In [36]:
@sayhello

Hello, World!!!


In [37]:
@sayhello "John"

Hello, John!!!


In [38]:
@sayhello begin
    1 + 3 
end

Hello, 4!!!


In [39]:
macro sayhello(x::Int)
    println("Calling Int ", x)
    return :(println("Hello Int, ", $x))
end

@sayhello (macro with 3 methods)

In [40]:
@sayhello 21

Calling Int 21
Hello Int, 21


In [41]:
x = 21
@sayhello x

Hello, 21!!!


In [42]:
macro intype(x)
    println(typeof(x))
end

@intype (macro with 1 method)

In [43]:
@intype begin
    x = 5
end

Expr


In [44]:
@intype x

Symbol


In [45]:
@intype 21

Int64


In [46]:
@intype "John"

String


## 8.3 Custom String Literals

Macros used to define custom string literals. `x_str` macro will be invoked on writing `x"...mystring..."`.  

In [47]:
m = match(r"a.a", "abracadabra")

RegexMatch("aca")

In [48]:
struct CapsString
    s::String
    CapsString(s::String)=new(uppercase(s))
end

macro C_str(s)
    CapsString(s)
end


@C_str (macro with 1 method)

In [49]:
C"This is Julia Upper Case String"

CapsString("THIS IS JULIA UPPER CASE STRING")

## 8.4 Generated Functions

Ability to define custom dispatch rules for functions. 

In [50]:
f(x::Integer) = x^2
f(x) = x

f (generic function with 2 methods)

In [51]:
f(4)

16

In [52]:
f("John")

"John"

In [53]:
@generated function genf(x)
    if x <: Integer 
        return :(x^2)
    else
        return :x
    end
end

genf (generic function with 1 method)

In [54]:
genf(4)

16

In [55]:
genf("John")

"John"

In [56]:
genf(3)

9

## 8.5 Commonly Used Macros

Julia language uses many common macros for ease of operations. Additionally, there are some macros that provide flexibility and readability to the code. 

### Source Locations

In [57]:
macro test()
    println(__module__)
    println(__source__)
end

@test (macro with 1 method)

In [58]:
@test

Main
#= In[58]:1 =#


In [59]:
function macro_usage()
    println("Dir:", @__DIR__, " file:", @__FILE__, " Line no:", @__LINE__)
end
macro_usage()

Dir:/home/sambit/work/books/HOPJ/Chapter-08 file:In[59] Line no:2


### eval

In [60]:
struct MyNumber
    v::Float64
end

Base.sin(m::MyNumber)=Base.sin(m.v)
Base.cos(m::MyNumber)=Base.cos(m.v)
Base.tan(m::MyNumber)=Base.tan(m.v)

In [61]:
for op in [:sin, :cos, :tan]
    eval(quote
            Base.$op(m::MyNumber)=Base.$op(m.v)
    end)
end


In [62]:
for op in [:sin, :cos, :tan]
    @eval Base.$op(m::MyNumber)=Base.$op(m.v)
end

### assert

In [63]:
macro myassert(ex, msg)
    return :($(ex) ? nothing : throw(AssertionError($msg)))
end
@myassert 1==0 "1 is not same as 0"

LoadError: AssertionError: 1 is not same as 0

In [64]:
macro myassert(ex, msgs...)
    msg = isempty(msgs) ? ex : msgs[1]
    msg = msg isa AbstractString ? String(msg) : string(msg)
    return :($(ex) ? nothing : throw(AssertionError($msg)))
end
@myassert 1==0

LoadError: AssertionError: 1 == 0

### time

In [65]:
@time begin
    sleep(0.3)
    1+1
end

  0.301730 seconds (15 allocations: 672 bytes)


2

In [66]:
macro mytime(ex)
    return quote
        local elapsedtime = time_ns()
        local val = $(ex)
        elapsedtime = time_ns() - elapsedtime
        println("Elapsed time: ", elapsedtime/1e9)
        val
    end
end

@mytime (macro with 1 method)

In [67]:
@mytime begin
    sleep(0.3)
    1+1
end

Elapsed time: 0.302202613


2

## 8.6 Conclusion

Metaprogramming is a very flexible way to program complex concepts in Julia with small amount of expressive code. While it improves readability and expressiveness of code, novice programmers may find it fairly advanced to begin with. They are an important aspect of programming paradigm in Julia. 

## Exercises